# Búsqueda hiperparámteros del modelo DNN 



- Cada bloque de 1000 días tiene su propia dinámica, volatilidad y características del mercado. Los hiperparámetros óptimos pueden variar de bloque a bloque.

- Sin embargo si pretendemos calcular hiperparámetros de cada bloque y de cada acción, los tiempos de cómputo se disparan.

- Por ello realizaremos búsqueda de hiperparámetros usando un bloque y una acción determinados. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

# Escalamiento
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Métricas
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error, r2_score

# modelo DNN
import tensorflow as tf
from tensorflow.keras import layers, regularizers


# hiperparametros
import keras_tuner as kt
from keras_tuner.tuners import RandomSearch 
from sklearn.model_selection import TimeSeriesSplit


## Lectura de datasets 

In [2]:
# Nombre de las acciones
tickers = [
    'TSCO', 'AZN', 'BARC', 'BP', 'BATS', 'HLMA',
    'HSBA', 'JMAT', 'LGEN', 'MKS', 'PSON', 'REL',
    'NWG', 'SHEL', 'SGE', 'SBRY', 'SDR', 'SVT',
    'SMIN', 'SSE', 'VOD'
]

data_frames = {} # Inicializo diciconario de data frames

for ticker in tickers: 
    
    df = pd.read_csv(f'datasets_features/{ticker}.csv')  # leo el dataset
    data_frames[ticker] = df                             # guardo dataframes
    
print(data_frames.keys())

dict_keys(['TSCO', 'AZN', 'BARC', 'BP', 'BATS', 'HLMA', 'HSBA', 'JMAT', 'LGEN', 'MKS', 'PSON', 'REL', 'NWG', 'SHEL', 'SGE', 'SBRY', 'SDR', 'SVT', 'SMIN', 'SSE', 'VOD'])


## Construyo build model 

In [3]:
def build_model(hp):
    model = tf.keras.Sequential()

    # Número de capas ocultas
    n_layers = hp.Int('n_layers', 1, 3)

    for i in range(n_layers):
        model.add(layers.Dense(
            units=hp.Choice(f'units_{i}', [32, 64, 128, 256]),
            activation=hp.Choice(f'activation_{i}', ['relu', 'tanh']),
            kernel_regularizer=regularizers.l2(
                hp.Choice(f'l2_{i}', [0.0, 0.001, 0.01])
            )
        ))
        model.add(layers.Dropout(
            hp.Choice(f'dropout_{i}', [0.1, 0.3, 0.5])
        ))

    # Capa de salida (regresión)
    model.add(layers.Dense(1))

    # Optimizador + learning rate
    lr = hp.Choice('learning_rate', [0.01, 0.001, 0.0001])
    optimizer_name = hp.Choice('optimizer', ['adam', 'rmsprop'])

    if optimizer_name == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=lr)
    else:
        opt = tf.keras.optimizers.RMSprop(learning_rate=lr)

    model.compile(
        optimizer=opt,
        loss='mse'
    )
    return model


In [4]:
# Funcion que dado un bloque de features me extrae el 75% de entrenamiento 
def train_global(df_bloque, return_estandarizado = True): 
    
    # Índice de separación
    N = len(df_bloque)
    split_idx = int(N * 0.75) 
    
    df_features = df_bloque.drop(columns=['Date', 'Open', 'High', 'Low','Close','Volume', 'prev_close', 'target']) # todas columnas r_i
    target      = df_bloque['target']  # columna del target
    
    X_train = df_features.iloc[ : split_idx]
    X_test  = df_features.iloc[split_idx : ]
    y_train = target.iloc[ : split_idx]
    y_test  = target.iloc[split_idx : ]

    # 2) ------------ Estandarización ---------------------
    # Features 
    scaler_x = MinMaxScaler(feature_range=(-1,1))
    X_train_s = scaler_x.fit_transform(X_train)     
    X_test_s  = scaler_x.transform(X_test)

    # Target (al estandarizar son 2D)
    scaler_y = MinMaxScaler(feature_range=(-1,1))
    y_train_s = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).ravel()    # Necesario reshape, scaler espera 2D      
    y_test_s  = scaler_y.transform(y_test.values.reshape(-1, 1))
    
    if return_estandarizado: 
        return X_train_s, y_train_s
    else: 
        return np.array(X_train), np.array(y_train)


## Busco los hiperparámetros con `RandomSearch`

In [5]:
# Elijo acción
df = data_frames['AZN']

# Defino el train que usará mi buscador de parámetros
X_train_global, y_train_global = train_global(df.iloc[0:1000], return_estandarizado=True) 
X_train_global.shape

(750, 20)

In [8]:
# 5 folds. X_train de 750 días.
#          Fold 1: [0:500] - [501:550]
#          Fold 2: [0:550] - [551:600]
#          Fold 3: [0:600] - [601:650]
#          Fold 4: [0:650] - [651:700]
#          Fold 5: [0:700] - [701:750]
                                            
tscv = TimeSeriesSplit(n_splits=5) 

# tuner prueba las diferentes combinaciones y guarda las mejores
tuner = RandomSearch(
    build_model,                       # funcion que construye modelo DNN con hp propuestos
    objective='val_loss',              # minimizamos error en validacion
    max_trials=20,                     # combinaciones que probamos
    executions_per_trial=1,            # no repetir configuracion 
    directory='dnn_tuning',            # carpeta donde se guardan resultados
    project_name='random_search_dnn_3'   # subcarpeta del proyecto
)


# Bucle para cada fold 
for train_idx, val_idx in tscv.split(X_train_global):
    X_tr, X_val = X_train_global[train_idx], X_train_global[val_idx]
    y_tr, y_val = y_train_global[train_idx], y_train_global[val_idx]
    
    # Ejecuto búsqueda en cada fold. Se guarda la mejor combinacion en cad fold. 
    tuner.search(
        X_tr, y_tr,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=32,
        callbacks=[tf.keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=5
        )]
    )

# Busca en TODOS los folds y TODOS los trials cuál configuración general obtuvo mejor rendimiento medio.
best_hps = tuner.get_best_hyperparameters(1)[0]
print(best_hps.values)

Reloading Tuner from dnn_tuning\random_search_dnn_3\tuner0.json
{'n_layers': 1, 'units_0': 32, 'activation_0': 'tanh', 'l2_0': 0.01, 'dropout_0': 0.5, 'learning_rate': 0.01, 'optimizer': 'rmsprop'}
